In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os
import pyarrow.feather as feather
from sklearn import preprocessing
from colorama import Fore, Back, Style
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator


In [3]:
#data=pd.read_feather("../Kaggle/AMEX/train_data.ftr")
data=pd.read_feather("../Kaggle/AMEX/test_data.ftr")

In [4]:
def make_data0(dat):

    #https://www.kaggle.com/code/cdeotte/xgboost-starter-0-793: pick up at most 2 from each sector
    #create interactions
    print("creating interactions")
    Varimp=["P_2","D_39","B_4","B_3","D_46","R_1","S_3","P_3"]
    temp=dat.copy()
    for i in range(7):
        for j in range(i+1,8):
            key=Varimp[i]+"_"+Varimp[j]
            temp.loc[:,key]=dat[Varimp[i]]*dat[Varimp[j]]
    del dat


    #create t
    print("creating t")
    temp["S_2"]=pd.to_datetime(temp["S_2"])
    temp["t"] = temp.groupby("customer_ID")["S_2"].apply(lambda g: g.max() - g)
    temp = temp.drop('S_2', 1)
    temp["t"]=temp["t"].dt.days

    return temp


def make_data1(temp):

    #declare column names
    cat_cols=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    num_cols=list(set(temp.columns) - set(cat_cols)-set(["customer_ID"]))


    #create per-ID data
    #numeric+category
    print("creating last")
    temp2=temp.groupby("customer_ID").ffill()
    temp2["customer_ID"]=temp["customer_ID"]
    last=temp2.groupby("customer_ID").last()
    last=last.add_suffix('_last')
    del temp2

    print("creating count")
    count=temp.groupby(temp.customer_ID)["t"].count()
    count=count.rename("count")

    output=pd.concat([count,last], axis=1)
    del last

    print("creating na_perc")
    na_perc=temp.drop('customer_ID', 1).isna().groupby(temp.customer_ID, sort=False).sum().div(count, axis='rows')
    del count

    na_perc=na_perc.add_suffix('_na_perc')
    output=pd.concat([output, na_perc], axis=1)
    del na_perc


    #category
    print("creating freq")
    def foo(x): m = pd.Series.mode(x); return m.values[0] if not m.empty else np.nan

    freq=temp.groupby("customer_ID")[cat_cols].agg(foo)
    freq=freq.add_suffix('_freq')

    output=pd.concat([output, freq], axis=1)
    del freq

    print("creating catchange")
    def change_cat(series): 
        bb=series.values
        count=sum((1 for i,x in enumerate(bb[:-1]) if (x!= bb[i+1]) and pd.isnull(x)==False))
        return count

    catchange=temp.groupby("customer_ID")[cat_cols].agg(change_cat)
    catchange=catchange.add_suffix('_catchange')

    output=pd.concat([output, catchange], axis=1)
    del catchange


    #numeric
    print("creating max")
    max=temp.groupby("customer_ID")[num_cols].apply(lambda g: g.max())
    max=max.add_suffix('_max')
    output=pd.concat([output, max], axis=1)
    del max

    print("creating min")
    min=temp.groupby("customer_ID")[num_cols].apply(lambda g: g.min())
    min=min.add_suffix('_min')
    output=pd.concat([output, min], axis=1)
    del min

    print("creating maxmin")
    maxmin=temp.groupby("customer_ID")[num_cols].apply(lambda g: (g.max()-g.min())/g.min())
    maxmin=maxmin.add_suffix('_maxmin')
    output=pd.concat([output, maxmin], axis=1)
    del maxmin

    print("creating mean")
    mean=temp.groupby("customer_ID")[num_cols].apply(lambda g: g.mean())
    mean=mean.add_suffix('_mean')
    output=pd.concat([output, mean], axis=1)
    del mean

    print("creating sd")
    sd=temp.groupby("customer_ID")[num_cols].apply(lambda g: np.std(g))
    sd=sd.add_suffix('_sd')
    output=pd.concat([output, sd], axis=1)
    del sd

    feather.write_feather(output, '../Kaggle/AMEX/output1.ftr')

    return output


def make_data2(temp):

    #declare column names
    cat_cols=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    num_cols=list(set(temp.columns) - set(cat_cols)-set(["customer_ID"]))

    #numeric - difference
    print("creating dif")
    dif=temp.groupby("customer_ID")[num_cols].ffill().diff()
    dif=dif.div(dif["t"], axis='rows')
    dif["customer_ID"]=temp["customer_ID"].tolist()

    print("creating maxdif")
    maxdif=dif.groupby("customer_ID")[num_cols].apply(lambda g: g.max())
    maxdif=maxdif.add_suffix('_maxdif')

    print("creating mindif")
    mindif=dif.groupby("customer_ID")[num_cols].apply(lambda g: g.min())
    mindif=mindif.add_suffix('_mindif')
    output=pd.concat([maxdif, mindif], axis=1)
    del maxdif,mindif

    print("creating maxmindif")
    maxmindif=dif.groupby("customer_ID")[num_cols].apply(lambda g: (g.max()-g.min())/g.min())
    maxmindif=maxmindif.add_suffix('_maxmindif')
    output=pd.concat([output, maxmindif], axis=1)
    del maxmindif,dif

    feather.write_feather(output, '../Kaggle/AMEX/output2.ftr')

    return output


def make_data3(output):

    #add features
    print("adding features")
    output["totalNA"]=output.isna().sum(axis=1).tolist()
    output["totalNA_count"]=output["totalNA"]*output["count"]


    #delete novariance
    print("original shape: ",output.shape)
    output=output.loc[:,(output != output.iloc[0]).any()]
    
    print("shape after deleting nonvariance: ",output.shape)

    #replace inf to nan
    output=output.replace([np.inf, -np.inf], np.nan)

    #delete all NAs
    output=output.dropna(axis=1, how='all')
    print("shape after deleting NAN: ",output.shape)


    #median/mode imputation
    cat_cols=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    cat_last=[s + "_last" for s in cat_cols]
    cat_freq=[s + "_freq" for s in cat_cols]
    cat=cat_freq + cat_last
    num=list(set(output.columns) - set(cat))

    output[cat]=output[cat].astype(str)

    print("Imputing by mode")
    imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median 
    imputer.fit(output[cat])
    imputer=imputer.transform(output[cat])
    newout_cat=pd.DataFrame(imputer, columns=output[cat].columns)

    imputer = SimpleImputer(missing_values=None,strategy='most_frequent')# strategy can also be mean or median 
    imputer.fit(newout_cat)
    imputer=imputer.transform(newout_cat)
    newout_cat=pd.DataFrame(imputer, columns=newout_cat.columns)

    print("Imputing by mean")
    imputer = SimpleImputer(strategy='mean')# strategy can also be mean or median 
    imputer.fit(output[num])
    imputer=imputer.transform(output[num])
    newout_cont=pd.DataFrame(imputer, columns=output[num].columns)

    output=pd.concat([newout_cat, newout_cont], axis=1)

    del newout_cat,newout_cont

    return output


creating interactions
creating t


/var/folders/n5/w7wbnl8956x5xtgwp11w9b2c0000gn/T/ipykernel_27334/2040686411.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  temp = temp.drop('S_2', 1)


In [5]:
temp=make_data0(data)
del data

output1=make_data1(temp)
output2=make_data2(temp)
del temp

#reset kernel

creating interactions
creating t


/var/folders/n5/w7wbnl8956x5xtgwp11w9b2c0000gn/T/ipykernel_29858/3276472079.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  temp = temp.drop('S_2', 1)


creating last
creating count
creating na_perc


/var/folders/n5/w7wbnl8956x5xtgwp11w9b2c0000gn/T/ipykernel_29858/3276472079.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  na_perc=temp.drop('customer_ID', 1).isna().groupby(temp.customer_ID, sort=False).sum().div(count, axis='rows')


creating freq
creating catchange
creating max
creating min
creating maxmin
creating mean


...

creating sd
creating dif


/var/folders/n5/w7wbnl8956x5xtgwp11w9b2c0000gn/T/ipykernel_29858/3276472079.py:126: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dif["customer_ID"]=temp["customer_ID"].tolist()


creating maxdif


: 

: 

In [3]:
#continue
output1=pd.read_feather("../Kaggle/AMEX/output1.ftr")
output2=pd.read_feather("../Kaggle/AMEX/output2.ftr")

In [4]:
output=pd.concat([output1, output2], axis=1)
del output1, output2

In [39]:
train_use=make_data3(output)

adding features
original shape:  (458913, 2107)
shape after deleting nonvariance:  (458913, 2035)
shape after deleting NAN:  (458913, 2033)
Imputing by mode
Imputing by mean


In [42]:
train_use.isna().sum().sum()
#[var for var in train_use.columns if train_use[var].isnull().sum() > 0]
del output



In [41]:
#feather.write_feather(train_use, '../Kaggle/AMEX/train_use.ftr')
feather.write_feather(train_use, '../Kaggle/AMEX/test_use.ftr')

In [223]:
#ensemble
test=pd.read_feather("../Kaggle/AMEX/test_data.ftr")
sub1 = pd.read_csv('../Kaggle/AMEX/submission_ens.csv')

In [278]:
#CatBoost prediction
col=data_use.columns
test_ID=test['customer_ID'].unique()
pred_catboost= clf.predict_proba(test_use[col])[:, 1]

In [280]:
sub2= pd.DataFrame({'customer_ID':test_ID, 'prediction2':pred_catboost})

In [283]:
#Ensemble
sub1 = pd.read_csv('../Kaggle/AMEX/submission_ens.csv')

blend = pd.merge(sub1, sub2, how='inner', on='customer_ID')
blend.prediction = (blend.prediction * 0.95 + blend.prediction2 * 0.05)
blend[['customer_ID', 'prediction']].to_csv('../Kaggle/AMEX/0613submission2.csv', index=False)